In [35]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer

In [36]:
t1 = pd.read_csv('art_lover.csv', header=0)
t2 = pd.read_csv('conservative.csv', header=0)
t3 = pd.read_csv('fashion_lover.csv', header=0)
t4 = pd.read_csv('food_lover.csv', header=0)
t5 = pd.read_csv('gamer.csv', header=0)
t6 = pd.read_csv('liberal.csv', header=0)
t7 = pd.read_csv('music_lover.csv', header=0)
t8 = pd.read_csv('religious.csv',header=0)
t9 = pd.read_csv('science_lover.csv', header=0)
t10 = pd.read_csv('sports_fan.csv', header=0)

In [37]:
t1['CATEGORY'] = "ART"
t2['CATEGORY'] = "CONSERVATIVE"
t3['CATEGORY'] = "FASHION"
t4['CATEGORY'] = "FOOD"
t5['CATEGORY'] = "GAMER"
t6['CATEGORY'] = "LIBERAL"
t7['CATEGORY'] = "MUSIC"
t8['CATEGORY'] = "RELIGIOUS"
t9['CATEGORY'] = "SCIENCE"
t10['CATEGORY'] = "SPORTS"

In [38]:
frames = [t1, t2, t3, t4, t5, t6, t7, t8, t9, t10]
tweets = pd.concat(frames, ignore_index = True)

In [39]:
def tweets_to_words(tweet):
    tweet = re.sub(r"http\S+", " ", tweet)

    text = BeautifulSoup(tweet).get_text() 
    
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
   
    words = letters_only.lower().split()                             

    stops = set(stopwords.words("english"))                  
    
    meaningful_words = [w for w in words if not w in stops]   

    return( " ".join( meaningful_words ))

In [90]:
num_tweets = tweets["text"].size
clean_tweets = []
for i in xrange( 0, num_tweets ):
    clean_tweets.append(tweets_to_words(tweets["text"][i]))
    if i %10000 == 0:
        print("PROCESSSING")

PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING
PROCESSSING


In [87]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 10000)
tfidf_transformer = TfidfTransformer()
X_train, X_test, y_train, y_test = train_test_split(clean_tweets, tweets['CATEGORY'], test_size=0.05, random_state=777)

In [26]:
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.fit_transform(X_test)

X_train_tfidf = tfidf_transformer.fit_transform(X_train_features)
X_test_tfidf = tfidf_transformer.transform(X_test_features)

In [27]:
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=100, random_state=42)
svm.fit(X_train_tfidf, y_train)

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=100, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False)

In [29]:
obs = np.array(y_test)
obs

array(['LIBERAL', 'GAMER', 'MUSIC', ..., 'LIBERAL', 'RELIGIOUS', 'GAMER'], dtype=object)

In [31]:
np.mean( pred == obs )

0.14977062738256769

In [74]:
def tweet_features(tweets):
    contains = []
    for tweet in tweets:
        features = {}
        for bigrams in nltk.bigrams(tweet.split(' ')):
            features['contains(%s)' % ','.join(bigrams)] = True
        contains.append(features)

    return contains

In [79]:
import nltk
clf = nltk.NaiveBayesClassifier
train = zip(tweet_features(clean_tweets), tweets['CATEGORY'])
test = zip(tweet_features(X_test), y_test)

In [82]:
clf = nltk.NaiveBayesClassifier.train(train)

In [83]:
clf.show_most_informative_features(20)

Most Informative Features
contains(thursday,night) = True           SPORTS : MUSIC  =   4277.6 : 1.0
contains(streaming,live) = True           SPORTS : SCIENC =   3279.2 : 1.0
contains(president,obama) = True           LIBERA : MUSIC  =   1891.1 : 1.0
contains(night,football) = True           SPORTS : RELIGI =   1814.3 : 1.0
  contains(live,twitter) = True           SPORTS : MUSIC  =   1463.2 : 1.0
  contains(donald,trump) = True           LIBERA : GAMER  =   1068.5 : 1.0
    contains(watch,game) = True           SPORTS : GAMER  =    903.4 : 1.0
      contains(see,work) = True              ART : GAMER  =    858.5 : 1.0
   contains(health,care) = True           LIBERA : MUSIC  =    528.6 : 1.0
contains(hillary,clinton) = True           CONSER : GAMER  =    464.8 : 1.0
      contains(born,day) = True              ART : MUSIC  =    350.4 : 1.0
contains(happy,birthday) = True              ART : SCIENC =    277.3 : 1.0
     contains(art,world) = True              ART : GAMER  =    269.0 : 1

In [84]:
print(nltk.classify.accuracy(clf, test))

0.968921625638


In [88]:
clf2 = nltk.NaiveBayesClassifier
train = zip(tweet_features(X_train), y_train)
test = zip(tweet_features(X_test), y_test)
clf2 = nltk.NaiveBayesClassifier.train(train)

In [89]:
print(nltk.classify.accuracy(clf2, test))

0.775681612611


In [101]:
X_train, X_test, y_train, y_test = train_test_split(clean_tweets, tweets['CATEGORY'], test_size=0.01, random_state=330)
clf3 = nltk.NaiveBayesClassifier
train = zip(tweet_features(X_train), y_train)
test = zip(tweet_features(X_test), y_test)
clf3 = nltk.NaiveBayesClassifier.train(train)

In [102]:
print(nltk.classify.accuracy(clf3, test))

0.790697674419


In [103]:
clean_tweets[0:4]

[u'learn changed game arthistory',
 u'natalia ryabova see natalia work artistoftheday',
 u'little goes long way',
 u'meet theotherartfair robin clare whose work explores jamaican pop culture']

In [105]:
X_train, X_test, y_train, y_test = train_test_split(tweets['text'], tweets['CATEGORY'], test_size=0.01, random_state=330)
clf4 = nltk.NaiveBayesClassifier
train = zip(tweet_features(X_train), y_train)
test = zip(tweet_features(X_test), y_test)
clf4 = nltk.NaiveBayesClassifier.train(train)

In [106]:
print(nltk.classify.accuracy(clf4, test))

0.762919896641


In [ ]:
import pickle
f = open('nb_clean_0.01.pickle', 'wb')
pickle.dump(clf3, f)
f.close()